# Numero de electores por circuito electoral.

Este notebook presenta un análisis del número de votantes por circuito electoral en Argentina, utilizando datos de las últimas elecciones nacionales. El análisis incluye el número de electores por circuito, así como el número de votos por cada tipo de elección, cargo y partido político.

## Contenidos

- Fuentes de datos y preparación
- Número de votantes por circuito
- Número de votos por tipo de elección
- Número de votos por cargo
- Número de votos por partido político


In [1]:
# Importing necessary libraries
import warnings
import pandas as pd
import glob

# Ignoring warnings
warnings.filterwarnings("ignore")
pd.options.display.max_rows = 99


## Mesas

In [2]:
import pandas as pd
df_mesas = pd.read_csv('./../datos/BD/mesas_table.csv')
# df_mesas = df_mesas.groupby(['eleccion_id', 'distrito_id', 'seccion_id', 'circuito_id', 'mesa_id'])['mesa_electores'].first().reset_index()
# Hay un problema con la columna mesa tipo, que falta completar en las elecciones 3, 9, 10
# Por eso, tiramos mesa tipo, y hacemos un groupby-first por el resto de las columnas, 

## Hay un problema en los datos originales de la eleccion de 2017. (8	2017	GENERAL	PROVISORIO	NORMAL)
## La solucion seria que dentro de un distrito, y circuito, los numeros de mesa de la eleccion (8	2017	GENERAL	PROVISORIO	NORMAL) serian los de la (7	2017	GENERAL	PROVISORIO	COMANDO)
fix_copy = df_mesas.loc[df_mesas.eleccion_id == 7].copy()
fix_copy['eleccion_id'] = 8

df_mesas = df_mesas.loc[df_mesas.eleccion_id != 8]
df_mesas = pd.concat([df_mesas, fix_copy])

## Revisar tambien problemas por los cuales hya mesas con cero votos en algunas elecciones.

In [4]:
df_mesas

,eleccion_id,distrito_id,seccion_id,circuito_id,mesa_id,mesa_electores,mesa_tipo
0,1,1,15,000161,6883,346.0,NATIVOS
1,1,1,15,000161,6901,346.0,NATIVOS
2,1,1,15,000161,6914,345.0,NATIVOS
3,1,1,15,000159,6813,346.0,NATIVOS
4,1,1,15,000162,6951,345.0,NATIVOS
...,...,...,...,...,...,...,...
452762,8,24,3,000023,462,42.0,NATIVOS
452763,8,24,3,000024,463,40.0,NATIVOS
452764,8,24,3,000025,464,22.0,NATIVOS
452765,8,24,3,000027,465,22.0,NATIVOS


In [5]:
# ### Para ver ese problema las siguientes tablitas. Parece que hay numeros de mesa de 2013 que por alguna razon se copiaron a 2017 pero no deberian. 

# import glob
# # Define the glob pattern to find the CSV files
# csv_votos_pattern = './../datos/Solo_ResultadosProvisorios/**/*.csv'

# # Find all the CSV files that match the glob pattern
# csv_votos_files = glob.glob(csv_votos_pattern, recursive=True)


# df = pd.DataFrame()
# for file in csv_votos_files:
#     print(file)
#     part = pd.read_csv(file)
#     part = part.loc[(part.distrito_id == 3)]
#     # concatenate
#     df = pd.concat([df, part], axis=0)


# table = df.loc[(df.distrito_id == 3) & (df.circuito_id == '0031')][['año', 'eleccion_tipo', 'padron_tipo', 'mesa_id']].drop_duplicates()
# table[1] = '1'

# table.set_index(['año', 'eleccion_tipo', 'padron_tipo', 'mesa_id']).unstack().fillna('')


# table = df.loc[(df.distrito_id == 3) & (df.circuito_id == '0058')][['año', 'eleccion_tipo', 'padron_tipo', 'mesa_id']].drop_duplicates()
# table[1] = '1'

# table.set_index(['año', 'eleccion_tipo', 'padron_tipo', 'mesa_id']).unstack().fillna('')

In [6]:
pd.options.display.float_format = '{:.1f}'.format

In [8]:
elecciones = pd.read_csv('./../datos/BD/eleccion_table.csv')
info = (df_mesas.merge(elecciones).groupby(['eleccion_id', 'eleccion_tipo', 'año', 'padron_tipo', 'distrito_id']).agg({'mesa_electores': 'sum'}).unstack()/1e4)

pd.options.display.max_columns = 30
def apply_gradient(df):
    return df.style.background_gradient(cmap='RdYlGn', vmin=0.5, vmax=1.5).format(' {:.2f}')

show = info.div(info.median(0), 1).round(2)

styled_df = apply_gradient(show)
styled_df

In [9]:
## Hay faltante importante con la de 2011 PASO, y a COMANDO 2019 le faltan algunas provincias

## Número de electores

In [35]:
## Agrupar por sección y tipo de elección
import os
if not os.path.exists('./../datos/out/n_electores_dpto.csv'):
    n_electores_dpto = df_mesas.groupby(['eleccion_id', 'distrito_id', 'seccion_id']
                                            ).agg({'mesa_electores': ['sum', 'size']}).reset_index()
    n_electores_dpto.to_csv('./../datos/out/n_electores_dpto.csv', index = False)

## Agrupar por circuito y tipo de elección
if not os.path.exists('./../datos/out/n_electores_circ.csv'):
    n_electores_circ = df_mesas.groupby(['eleccion_id', 'distrito_id', 'seccion_id', 'circuito_id']
                                            ).agg({'mesa_electores': ['sum', 'size']}).reset_index()
    n_electores_circ.to_csv('./../datos/out/n_electores_circ.csv', index = False)


## Número de votos por tipo de voto

In [32]:
import glob
import pandas as pd

csv_votos = glob.glob('./../datos/BD/votos_eleccion_*_table.csv', recursive=True)
usecols = ['distrito_id', 'votos_tipo', 'mesa_id', 'cargo_id', 'circuito_id', 'seccion_id', 'seccionprovincial_id', 'votos_cantidad', 'eleccion_id']

df = pd.DataFrame()
for file in csv_votos:
    part = pd.read_csv(file, usecols=usecols)
    print(file, part.shape, part.votos_cantidad.sum())

    if file == './../datos/BD/votos_eleccion_8_table.csv':
        ## Necesitamos hacer este merge para transladar la correccion de mesas "duplicadas" de 2017 del dataset de mesas al de votos
        part = df_mesas.loc[df_mesas.eleccion_id == 8][['eleccion_id', 'distrito_id', 'seccion_id', 'circuito_id', 'mesa_id']].merge(part.fillna(0), how = 'left')   ## Elecciones 6, 3, 10, 2, 16 tienen algo con este merge...
        print(file, part.shape, part.votos_cantidad.sum())

    # concatenate
    df = pd.concat([df, part])

./../datos/BD/votos_eleccion_5_table.csv (5053072, 9) 144522104
./../datos/BD/votos_eleccion_8_table.csv (2438892, 9) 61800457
./../datos/BD/votos_eleccion_8_table.csv (2108825, 9) 56526779
./../datos/BD/votos_eleccion_1_table.csv (2116876, 9) 37294635
./../datos/BD/votos_eleccion_4_table.csv (565020, 9) 25738560
./../datos/BD/votos_eleccion_6_table.csv (8956666, 9) 128463868
./../datos/BD/votos_eleccion_11_table.csv (264145, 9) 48043
./../datos/BD/votos_eleccion_3_table.csv (2277250, 9) 35291518
./../datos/BD/votos_eleccion_15_table.csv (230717, 9) 24960
./../datos/BD/votos_eleccion_12_table.csv (3159297, 9) 95811610
./../datos/BD/votos_eleccion_10_table.csv (4807171, 9) 49410512
./../datos/BD/votos_eleccion_0_table.csv (3503909, 9) 84365358
./../datos/BD/votos_eleccion_14_table.csv (5082019, 9) 89203585
./../datos/BD/votos_eleccion_2_table.csv (1697141, 9) 45336513
./../datos/BD/votos_eleccion_13_table.csv (303176, 9) 57822
./../datos/BD/votos_eleccion_9_table.csv (153773, 9) 88203
.

In [27]:
# Debug
# csvs = ['./../datos/BD/votos_eleccion_3_table.csv',
# './../datos/BD/votos_eleccion_10_table.csv',
# './../datos/BD/votos_eleccion_2_table.csv',
# './../datos/BD/votos_eleccion_16_table.csv',
# './../datos/BD/votos_eleccion_6_table.csv']


# for file in csvs:
#     part = pd.read_csv(file)
#     print(file, part.shape)
#     ## Necesitamos hacer este merge para transladar la correccion de mesas "duplicadas" de 2017 del dataset de mesas al de votos
#     part1 = df_mesas[['eleccion_id', 'distrito_id', 'seccion_id', 'circuito_id', 'mesa_id']].merge(part, how = 'left').dropna()
#     part2 = df_mesas[['eleccion_id', 'distrito_id', 'seccion_id', 'circuito_id', 'mesa_id']].merge(part, how = 'left').dropna().drop_duplicates()
#     part3 = df_mesas[['eleccion_id', 'distrito_id', 'seccion_id', 'circuito_id', 'mesa_id']].drop_duplicates().merge(part, how = 'left').dropna()
#     print(file, part1.shape)
#     print(file, part2.shape)
#     print(file, part3.shape)

./../datos/BD/votos_eleccion_3_table.csv (2277250, 11)
./../datos/BD/votos_eleccion_3_table.csv (1477150, 11)
./../datos/BD/votos_eleccion_3_table.csv (1477150, 11)
./../datos/BD/votos_eleccion_3_table.csv (1477150, 11)
./../datos/BD/votos_eleccion_10_table.csv (4807171, 11)
./../datos/BD/votos_eleccion_10_table.csv (3805475, 11)
./../datos/BD/votos_eleccion_10_table.csv (3805475, 11)
./../datos/BD/votos_eleccion_10_table.csv (3805475, 11)
./../datos/BD/votos_eleccion_2_table.csv (1697141, 11)
./../datos/BD/votos_eleccion_2_table.csv (0, 11)
./../datos/BD/votos_eleccion_2_table.csv (0, 11)
./../datos/BD/votos_eleccion_2_table.csv (0, 11)
./../datos/BD/votos_eleccion_16_table.csv (5214490, 11)


KeyboardInterrupt: 

In [34]:
## Para Votos Tipo
# Agrupar por: 'cargo_id', 'seccion_id', 'distrito_id', 'votos_tipo'
# Despues agregar circuito id, despues agregar tambien mesa_id, para tener las tablas por circuito y mesa

import os
## Agrupar por sección y tipo de elección
if not os.path.exists('./../datos/out/votos_tipo_dpto.csv'):
    votos_tipo_dpto = df.groupby(['eleccion_id', 'cargo_id', 'distrito_id', 'seccionprovincial_id', 'seccion_id', 'votos_tipo']
                                            ).agg({'votos_cantidad': 'sum'}).reset_index()
    votos_tipo_dpto.to_csv('./../datos/out/votos_tipo_dpto.csv', index = False)

## Agrupar por circuito y tipo de elección
if not os.path.exists('./../datos/out/votos_tipo_circ.csv'):
    votos_tipo_circ = df.groupby(['eleccion_id', 'cargo_id', 'distrito_id', 'seccionprovincial_id', 'seccion_id', 'circuito_id', 'votos_tipo']
                                            ).agg({'votos_cantidad': 'sum'}).reset_index()
    votos_tipo_circ.to_csv('./../datos/out/votos_tipo_circ.csv', index = False)

## Agrupar por mesa y tipo de elección
if not os.path.exists('./../datos/out/votos_tipo_mesa.csv'):
    votos_tipo_mesa = df.groupby(['eleccion_id', 'cargo_id', 'distrito_id', 'seccionprovincial_id', 'seccion_id', 'circuito_id', 'mesa_id', 'votos_tipo']
                                            ).agg({'votos_cantidad': 'sum'}).reset_index()
    votos_tipo_mesa.to_csv('./../datos/out/votos_tipo_mesa.csv', index = False)

    # , 'agrupacion_id', 'lista_numero'
